# Objetivo de la API
1. Objetivo: Ayudar a los conductores a decidir en qué zonas operar según la cantidad de trabajo que buscan:
    - Mucho trabajo: Zonas con alta demanda de viajes.
    -  Algo de trabajo: Zonas con demanda media.
    - Poco trabajo: Zonas con baja demanda, pero con potenciales beneficios específicos (menos competencia, distancias largas, etc.).
2. Datos clave para el análisis:
    - Viajes: Información sobre cuántos viajes ocurren en cada zona (demanda).
    - Ganancias promedio: Cuánto ganan los conductores por viaje en cada zona.
    - Distancia promedio: Para entender la relación entre distancia recorrida y ganancias.
    - Ubicaciones geográficas: Para facilitar la navegación hacia zonas recomendadas mediante un enlace a Google Maps.

# Beneficios para el negocio:
1. Optimización del tiempo del conductor:
    - Ayuda a los conductores a concentrarse en zonas donde tendrán más viajes o mejores ganancias según su preferencia.
2. Maximización de ingresos:
    - Identifica zonas con mayores ganancias promedio, lo que ayuda a los conductores a aumentar sus ingresos por hora.
3. Uso de datos geográficos:
    - El enlace a Google Maps permite una navegación sencilla, facilitando la llegada a zonas específicas.
4. Flexibilidad:
    - Los conductores pueden elegir entre priorizar cantidad de viajes, ganancias por viaje, o moverse a zonas menos saturadas.

# Caso práctico:

Un conductor consulta la API con /recommendations?work_amount=mucho. El sistema le recomienda zonas con alta demanda, junto con datos como ganancias promedio y un enlace directo a Google Maps. Esto le permite dirigirse directamente a las zonas más rentables y mejorar su eficiencia.

In [34]:
from fastapi import FastAPI
import pandas as pd
import numpy as np

# Load datasets
taxi_data_path = 'Yellow_Tripdata_2024-10_reducido.csv'
zone_data_path = 'transformed_taxi_zone_merged_with_locations.csv'

taxi_data = pd.read_csv(taxi_data_path)
zone_data = pd.read_csv(zone_data_path)

# Create FastAPI app
app = FastAPI()

# Preprocess data
# Merge dataframes if needed
data = taxi_data.merge(zone_data, left_on='PULocationID', right_on='locationid_x', how='left')

def calculate_recommendations(work_amount):
    # Define demand ranges based on work amount
    demand_filters = {
        "mucho": lambda df: df[df['trip_count'] >= df['trip_count'].quantile(0.75)],  # High demand
        "algo": lambda df: df[(df['trip_count'] >= df['trip_count'].quantile(0.25)) & (df['trip_count'] < df['trip_count'].quantile(0.75))],  # Median demand
        "poco": lambda df: df[df['trip_count'] < df['trip_count'].quantile(0.25)]  # Low demand
    }

    # Group by zones and calculate average earnings, trips, and distance
    zone_summary = (
        data.groupby('zone')
        .agg({
            'fare_amount': 'mean',
            'trip_distance': 'mean',
            'PULocationID': 'count',
            'zone_latitude': 'mean',
            'zone_longitude': 'mean'
        })
        .rename(columns={
            'fare_amount': 'avg_earning',
            'trip_distance': 'avg_distance',
            'PULocationID': 'trip_count'
        })
        .reset_index()
    )

    # Filter zones based on demand level
    demand_filter = demand_filters.get(work_amount.lower(), lambda df: df)
    filtered_zones = demand_filter(zone_summary)

    # Sort by trip count and average earnings, and select top zones
    recommended_zones = filtered_zones.sort_values(by=['trip_count', 'avg_earning'], ascending=False).head(5)

    # Add Google Maps URL using latitude and longitude
    recommended_zones['maps_url'] = recommended_zones.apply(
        lambda row: f"https://www.google.com/maps?q={row['zone_latitude']},{row['zone_longitude']}", axis=1
    )

    return recommended_zones[['zone', 'avg_earning', 'avg_distance', 'trip_count', 'maps_url']].to_dict(orient='records')

@app.get("/recommendations")
def get_recommendations(work_amount: str):
    """
    Get zone recommendations based on desired work amount.
    work_amount: "mucho", "algo", or "poco".
    """
    try:
        recommendations = calculate_recommendations(work_amount)
        return {"status": "success", "data": recommendations}
    except Exception as e:
        return {"status": "error", "message": str(e)}


C:\Users\jerom\AppData\Local\Temp\ipykernel_15944\824333614.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  taxi_data = pd.read_csv(taxi_data_path)


In [38]:
get_recommendations("Mucho")

{'status': 'success',
 'data': [{'zone': 'JFK Airport',
   'avg_earning': 59.36172296412674,
   'avg_distance': 15.831527101335428,
   'trip_count': 19095,
   'maps_url': 'https://www.google.com/maps?q=40.90406018978221,-73.98946405903479'},
  {'zone': 'Upper East Side South',
   'avg_earning': 13.404588835692113,
   'avg_distance': 1.9072988898826748,
   'trip_count': 19007,
   'maps_url': 'https://www.google.com/maps?q=40.82481644952361,-74.00010353638103'},
  {'zone': 'Midtown Center',
   'avg_earning': 16.922668123739072,
   'avg_distance': 2.4220712844653667,
   'trip_count': 17844,
   'maps_url': 'https://www.google.com/maps?q=40.74862425294619,-73.65453220858619'},
  {'zone': 'Upper East Side North',
   'avg_earning': 13.597059493822718,
   'avg_distance': 5.668473671584503,
   'trip_count': 16674,
   'maps_url': 'https://www.google.com/maps?q=40.87324570255901,-73.53026196122036'},
  {'zone': 'Midtown East',
   'avg_earning': 16.33640629662787,
   'avg_distance': 2.353503431811